# WFI Reference Pixel Reference File Pipeline Example

This notebook shows how to use the WFI Reference File Pipeline to create reference pixel reference files to submit to CRDS. At this stage, it is in the testing phase, and this notebook is used to assemble the steps of the pipeline to make test data.

Presently, the input files come from TVAC1 .asdf files. These data are located on central storage at:

```
/grp/roman/jsanchez/TVAC1_test_files/
```

# Some imports ... 

In [1]:
from wfi_reference_pipeline.utilities import logging_functions
logging_functions.configure_logging('wfi_refpixel_example')

from wfi_reference_pipeline.referencepixel import referencepixel
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta

import asdf
import glob
import logging
import matplotlib.pyplot as plt
import os
import h5py

# Reference Pixel Files

To make a reference pixel reference file, we need to perform the Improved Roman Reference Correction (Rauscher et al., in prep), We'll use several darks to compute a model of the read noise in the normal pixels that is a linear combination of the reference output, left, and right column pixels (IRRC; Rauscher et al., in prep).  The sums for each exposure are then combined/summed together to create a final model that minimizes the 1/f noise given as
        $Fn = \alpha Fa + \gamma Fl + \zeta Fr$.  
 The coefficients are then saved as a final reference class attribute.  

In [2]:
# Set the directory of the TVAC1 data.
tvac1_dir = '/grp/roman/jsanchez/TVAC1_test_files/NOM_OPS/OTP00639_All_TV1a_R1_MCEB'
logging.info(f'Using directory {tvac1_dir} for TVAC1 data.')

In [3]:
# Grab the files.
dark_files = glob.glob(os.path.join(tvac1_dir, '*WFI01*.asdf'))[0:2]

# Record file names in log.
logging.info('Files used for reference pixel creation:')
for df in dark_files:
    logging.info(f'\t{os.path.basename(df)}')
  

## Setup 

The example refpix reference file will be created in the RFP scratch directory. This directory has it contents automatically deleted every night. A default outfile name is given and if clobber set to True will be overwritten.

**tmppath**: The intermediatary files will be temporarily saved during the IRRC.  
- state the path for temporary sums to save in. The path must fully exist.  It will create a folder inside called 'tmp_IRRC_[detector]'
- can use tmppath = None, and the tmp_IRRC_[detector] will save in the current working directory

**outfile**: filename for reference pixel coefficient file

In [4]:
sca = 'WFI01'  # state which detector is being used.

write_path = '/grp/roman/RFP/DEV/scratch/'  # Set the write path to be in the RFP scratch directory.
tmppath = write_path # use the same folder to save the temporary sums folder
outfile = write_path + f'roman_dev_refpix_{sca}.asdf' # Default reference pixel development filename.

# Meta Data

The RFP utilizes python data classes to construct reference file type dependent meta data for development from the "ref_type" keyword, which is then exported into a dictionary for valid usage with roman data models. 

In [5]:
logging.info("Generating ReferencePixel Metadata")
tmp = MakeDevMeta(ref_type='REFPIX')  # Get the default reference file type specific development meta data.
refpixel_dev_meta = tmp.meta_referencepixel.export_asdf_meta()  # Export the meta data as a dictionary.

In [6]:
print(refpixel_dev_meta)  # Inspect default development meta data.

{'reftype': 'REFPIX', 'pedigree': 'DUMMY', 'description': 'For RFP Development and DMS Build Support.', 'author': 'RFP', 'useafter': <Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, 'telescope': 'ROMAN', 'origin': 'STSCI', 'instrument': {'name': 'WFI', 'detector': 'WFI01'}, 'input_units': Unit("DN"), 'output_units': Unit("DN")}


In [7]:
refpixel_dev_meta.update({'useafter': '2023-01-01T00:00:00.000'})  # Update to the useafter date found on CRDS to replace an existing file.
# Or make a new useafter date.

refpixel_dev_meta['instrument'].update({'detector': sca})

In [8]:
# add list of files that will make weights to metadata
refpixel_dev_meta['irrc_weight_filenames']= dark_files


In [9]:
print(refpixel_dev_meta)  # Inspect updated meta data.

{'reftype': 'REFPIX', 'pedigree': 'DUMMY', 'description': 'For RFP Development and DMS Build Support.', 'author': 'RFP', 'useafter': '2023-01-01T00:00:00.000', 'telescope': 'ROMAN', 'origin': 'STSCI', 'instrument': {'name': 'WFI', 'detector': 'WFI01'}, 'input_units': Unit("DN"), 'output_units': Unit("DN"), 'irrc_weight_filenames': ['/grp/roman/jsanchez/TVAC1_test_files/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021080847_WFI01_uncal.asdf', '/grp/roman/jsanchez/TVAC1_test_files/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021083839_WFI01_uncal.asdf']}


# Make the referencepixel coefficients via the ReferencePixel class

We now have the files and folder paths of darks for one detector. We can pass this list of images to the ReferencePixel class to make ASDF reference pixel reference files for Roman.

Note: this step is relatively slow.  Be patient.

In [10]:
 
refpix_ref = referencepixel.ReferencePixel(dark_files,  outfile = outfile,
                                           meta_data=refpixel_dev_meta)
refpix_ref.make_referencepixel_coeffs(tmppath=tmppath)


## Build the datamodel for the reference file, write it to disk, and change the permissions of the file. 

In [11]:
# save to .asdf file
refpix_ref.save_referencepixel()
os.chmod(outfile, 0o666)  # Set file permissions to read+write for owner, group, and global.
print('Made file -> ', outfile)
logging.info(f'outfile created: {outfile}')

Made file ->  /grp/roman/RFP/DEV/scratch/roman_dev_refpix_WFI01.asdf
